In [1]:
#%% 赛题目的：预测单位销量(unit_sales)
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import swifter
import polars as pl
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from datetime import date,datetime,timedelta    

In [2]:
#%% 数据读取
path = '/Users/ziyan/Documents/机器学习/机器学习比赛/favorita-grocery-sales-forecasting/'

# 大文件用polars读再转swifter ，节省一半时间
time2 = time.time()
train = pl.read_csv(path+'train.csv',dtypes={'date': pl.Datetime,'onpromotion':bool,'unit_sales':float})
train = train.to_pandas()
train['unit_sales'] = train['unit_sales'].swifter.apply(lambda u:np.log1p(u) if u>0 else 0.0)
print('2:',time.time() - time2)

test = pd.read_csv(path+"test.csv")
transactions = pd.read_csv(path+"transactions.csv")
items = pd.read_csv(path+"items.csv")
stores = pd.read_csv(path+"stores.csv")
oil = pd.read_csv(path+"oil.csv")
holidays_events = pd.read_csv(path+"holidays_events.csv")


Dask Apply:   0%|          | 0/17 [00:00<?, ?it/s]

/Users/ziyan/opt/anaconda3/lib/python3.9/site-packages/dask/utils.py:1241: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/Users/ziyan/opt/anaconda3/lib/python3.9/site-packages/dask/utils.py:1241: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/Users/ziyan/opt/anaconda3/lib/python3.9/site-packages/dask/utils.py:1241: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)


2: 52.03027272224426


In [3]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,2.079442,None
1,1,2013-01-01,25,105574,0.693147,None
2,2,2013-01-01,25,105575,1.098612,None
3,3,2013-01-01,25,108079,0.693147,None
4,4,2013-01-01,25,108701,0.693147,None


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[us]
 2   store_nbr    int64         
 3   item_nbr     int64         
 4   unit_sales   float64       
 5   onpromotion  object        
dtypes: datetime64[us](1), float64(1), int64(3), object(1)
memory usage: 5.6+ GB


In [5]:
train.describe()

,id,date,store_nbr,item_nbr,unit_sales
count,1.254970e+08,125497040,1.254970e+08,1.254970e+08,1.254970e+08
mean,6.274852e+07,2015-09-15 16:43:18.789250,2.746458e+01,9.727692e+05,1.750856e+00
min,0.000000e+00,2013-01-01 00:00:00,1.000000e+00,9.699500e+04,0.000000e+00
25%,3.137426e+07,2014-09-20 00:00:00,1.200000e+01,5.223830e+05,1.098612e+00
50%,6.274852e+07,2015-11-21 00:00:00,2.800000e+01,9.595000e+05,1.609438e+00
75%,9.412278e+07,2016-10-17 00:00:00,4.300000e+01,1.354380e+06,2.302585e+00
max,1.254970e+08,2017-08-15 00:00:00,5.400000e+01,2.127114e+06,1.140133e+01
std,3.622788e+07,NaN,1.633051e+01,5.205336e+05,8.815504e-01


## 构建描述性统计DataFrame

In [6]:
stats = []
for col in train.columns:
    stats.append((col, #特征
                  train[col].nunique(), #属性个数
                  round(train[col].isnull().sum() * 100 / train.shape[0], 3), #缺失值占比
                  round(train[col].value_counts(normalize=True, dropna=False).values[0] * 100,3), #最大属性占比
                  train[col].dtype))#特征类型
    #注意这里的append是append了一个元组
#学习下面tuple->list->dataframe的过程 
stats_df = pd.DataFrame(stats, columns=['特征', '属性个数', '缺失值占比', '最大属性占比', '特征类型'])
stats_df.sort_values('缺失值占比', ascending=False)[:10]

,特征,属性个数,缺失值占比,最大属性占比,特征类型
5,onpromotion,2,17.257,76.519,object
0,id,125497040,0.000,0.000,int64
1,date,1684,0.000,0.094,datetime64[us]
2,store_nbr,54,0.000,2.799,int64
3,item_nbr,4036,0.000,0.067,int64
4,unit_sales,257719,0.000,18.682,float64
